# 5.2 NLP

## 本節の準備

```
(pydataenv) $ pip install mecab-python3
(pydataenv) $ pip install gensim
```

# 5.2.2 形態素解析

In [1]:
import MeCab
text = "吾輩は猫である"
# 形態素解析の結果をChasenの形式で表示
t = MeCab.Tagger('-Ochasen')
result = t.parse(text)
print(result)

吾輩	ワガハイ	吾輩	名詞-代名詞-一般		
は	ハ	は	助詞-係助詞		
猫	ネコ	猫	名詞-一般		
で	デ	だ	助動詞	特殊・ダ	連用形
ある	アル	ある	助動詞	五段・ラ行アル	基本形
EOS



In [2]:
# 形態素解析の結果を確認
result

'吾輩\tワガハイ\t吾輩\t名詞-代名詞-一般\t\t\nは\tハ\tは\t助詞-係助詞\t\t\n猫\tネコ\t猫\t名詞-一般\t\t\nで\tデ\tだ\t助動詞\t特殊・ダ\t連用形\nある\tアル\tある\t助動詞\t五段・ラ行アル\t基本形\nEOS\n'

In [3]:
# 形態素解析の結果を、改行を区切りとして行ごとに分割
results = result.splitlines()
# EOSの行は対象外にする
for res in results[:-1]:
    # タブ区切りとして各要素に分割
    res_split = res.split('\t')
    print(res_split)

['吾輩', 'ワガハイ', '吾輩', '名詞-代名詞-一般', '', '']
['は', 'ハ', 'は', '助詞-係助詞', '', '']
['猫', 'ネコ', '猫', '名詞-一般', '', '']
['で', 'デ', 'だ', '助動詞', '特殊・ダ', '連用形']
['ある', 'アル', 'ある', '助動詞', '五段・ラ行アル', '基本形']


# 5.2.3 Bag of Words (BoW)

In [4]:
import MeCab

documents = ['子供が走る', '車が走る', '子供の脇を車が走る']

words_list = []

# 形態素解析の結果をChasenの出力形式で表示
t = MeCab.Tagger('-Ochasen')
# 各文に形態素解析を実行
for s in documents:
    s_parsed = t.parse(s)
    words_s = []
    # 各文の形態素をリストにまとめる
    for line in s_parsed.splitlines()[:-1]:
        words_s.append(line.split('\t')[0])
    words_list.append(words_s)
    
print(words_list)

[['子供', 'が', '走る'], ['車', 'が', '走る'], ['子供', 'の', '脇', 'を', '車', 'が', '走る']]


In [5]:
# 生成する辞書
word2int = {}
i = 0
# 各文書の単語のリストに対して処理を反復
for words in words_list:
    # 文書内の各単語に対して処理を反復
    for word in words:
        # 単語が辞書に含まれていなければ追加して対応する整数を割り当てる
        if word not in word2int:
            word2int[word] = i
            i += 1
print(word2int)

{'子供': 0, 'が': 1, '走る': 2, '車': 3, 'の': 4, '脇': 5, 'を': 6}


In [6]:
import numpy as np
# BoWを計算し、文書×単語の行列を生成
bow = np.zeros((len(words_list), len(word2int)), dtype=np.int)
# 各行の単語を抽出し端との出現回数をカウント
for i, words in enumerate(words_list):
    for word in words:
        bow[i, word2int[word]] += 1
print(bow)

[[1 1 1 0 0 0 0]
 [0 1 1 1 0 0 0]
 [1 1 1 1 1 1 1]]


In [7]:
import pandas as pd
pd.DataFrame(bow, columns=list(word2int))

,子供,が,走る,車,の,脇,を
0,1,1,1,0,0,0,0
1,0,1,1,1,0,0,0
2,1,1,1,1,1,1,1


In [8]:
from gensim import corpora
# 辞書を作成する
word2int_gs = corpora.Dictionary(words_list)
print(word2int_gs)

Dictionary(7 unique tokens: ['が', '子供', '走る', '車', 'の']...)


In [9]:
# 単語と整数の対応
print(word2int_gs.token2id)

{'が': 0, '子供': 1, '走る': 2, '車': 3, 'の': 4, 'を': 5, '脇': 6}


In [10]:
# 1番目の文書に含まれる単語の出現回数をカウント
print(word2int_gs.doc2bow(words_list[0]))

[(0, 1), (1, 1), (2, 1)]


In [11]:
import numpy as np
from gensim import matutils
# Bag of Words を計算し、文書×単語の行列を生成
bow_gs = np.array(
                    [matutils.corpus2dense(
                        [word2int_gs.doc2bow(words)],
                            num_terms=len(word2int)).T[0]
                                for words in words_list]
            ).astype(np.int)
print(bow_gs)

[[1 1 1 0 0 0 0]
 [1 0 1 1 0 0 0]
 [1 1 1 1 1 1 1]]


In [12]:
# pandasのデータフレームに変換
bow_gs_df = pd.DataFrame(bow_gs, 
                                        columns=list(word2int_gs.values()))
bow_gs_df

,が,子供,走る,車,の,を,脇
0,1,1,1,0,0,0,0
1,1,0,1,1,0,0,0
2,1,1,1,1,1,1,1


In [13]:
# 単語をスペース区切りで並べた文を生成
words_split = np.array([' '.join(words) for words in words_list])
print(words_split)

['子供 が 走る' '車 が 走る' '子供 の 脇 を 車 が 走る']


In [14]:
from sklearn.feature_extraction.text import CountVectorizer

# Bag of Wordsを計算
vectorizer = CountVectorizer(token_pattern=u'(?u)\\b\\w+\\b')
bow_vec = vectorizer.fit_transform(words_split)
# NumPy配列に変換
bow_vec.toarray()

array([[1, 0, 0, 1, 0, 1, 0],
       [1, 0, 0, 0, 0, 1, 1],
       [1, 1, 1, 1, 1, 1, 1]], dtype=int64)

In [15]:
vectorizer.get_feature_names()

['が', 'の', 'を', '子供', '脇', '走る', '車']

# 5.2.4 TF-IDF

In [16]:
bow_gs_df

,が,子供,走る,車,の,を,脇
0,1,1,1,0,0,0,0
1,1,0,1,1,0,0,0
2,1,1,1,1,1,1,1


In [17]:
# TFとしてBoWを使用
tf = bow_gs

In [18]:
print(tf)

[[1 1 1 0 0 0 0]
 [1 0 1 1 0 0 0]
 [1 1 1 1 1 1 1]]


In [19]:
# IDFを計算
idf = np.log((bow_gs.shape[0] + 1)/
                (np.sum(bow_gs, axis=0, keepdims=0) + 1))
print(idf)

[0.         0.28768207 0.         0.28768207 0.69314718 0.69314718
 0.69314718]


In [20]:
# TF-IDFを計算
tf_idf = tf * (idf + 1)
tf_idf_normalized = tf_idf / np.sqrt(np.sum(tf_idf**2, axis=1, keepdims=True))
print(tf_idf_normalized)

[[0.52284231 0.67325467 0.52284231 0.         0.         0.
  0.        ]
 [0.52284231 0.         0.52284231 0.67325467 0.         0.
  0.        ]
 [0.26806191 0.34517852 0.26806191 0.34517852 0.45386827 0.45386827
  0.45386827]]


In [21]:
from sklearn.feature_extraction.text import TfidfTransformer
# TfidfTransformerクラスをインスタンス化
tfidf = TfidfTransformer(use_idf=True, norm='l2', smooth_idf=True)
# TF-IDFを算出
print(tfidf.fit_transform(bow_gs).toarray())

[[0.52284231 0.67325467 0.52284231 0.         0.         0.
  0.        ]
 [0.52284231 0.         0.52284231 0.67325467 0.         0.
  0.        ]
 [0.26806191 0.34517852 0.26806191 0.34517852 0.45386827 0.45386827
  0.45386827]]


# 5.2.5 極性判定

In [22]:
import zipfile
import urllib.request

# 青空文庫の「吾輩は猫である」のファイルをダウンロード
urllib.request.urlretrieve('https://www.aozora.gr.jp/cards/000148/files/789_ruby_5639.zip', '789_ruby_5639.zip')

# zipファイルを解凍しデータを読み込む
with zipfile.ZipFile('789_ruby_5639.zip', 'r') as zipf:
    data = zipf.read('wagahaiwa_nekodearu.txt')  # bytesを変換
text = data.decode('shift_jis')  # shift-jisに変換

In [23]:
import re
# ルビ　, 注釈, 改行コードを除去
text = re.split(r'\-{5,}', text)[2]
text = text.split('底本: ')[0]
text = re.sub(r'《.+?》', '', text)
text = re.sub(r'［＃.+?］', '', text)
text = text.strip()
# 空白文字を除去
text = text.replace('\u3000', '')
# 改行コードを除去
text = text.replace('\r', '').replace('\n', '')
# 「。」を区切り文字として分割
sentences = text.split('。')
print(sentences[:5])

['一吾輩は猫である', '名前はまだ無い', 'どこで生れたかとんと見当がつかぬ', '何でも薄暗いじめじめした所でニャーニャー泣いていた事だけは記憶している', '吾輩はここで始めて人間というものを見た']


In [24]:
import MeCab

words_list = []

# MeCabにより形態素解析を実行
t = MeCab.Tagger('-Ochasen')
# 各文に形態素解析を実行(最後の要素は単語がないため除外)
for sentence in sentences[:-1]:
    sentence_parsed =t.parse(sentence)
    word_s = []
    # 各文書に現れる単語のリストに対して処理を反復
    for line in sentence_parsed.splitlines()[:-1]:
        word_s.append(line.split('\t')[2])
    words_list.append(word_s)

print(words_list[:10])

[['一', '吾輩', 'は', '猫', 'だ', 'ある'], ['名前', 'は', 'まだ', '無い'], ['どこ', 'で', '生れる', 'た', 'か', 'とんと', '見当', 'が', 'つく', 'ぬ'], ['何', 'でも', '薄暗い', 'じめじめ', 'する', 'た', '所', 'で', 'ニャーニャー', '泣く', 'て', 'いた事', 'だけ', 'は', '記憶', 'する', 'て', 'いる'], ['吾輩', 'は', 'ここ', 'で', '始める', 'て', '人間', 'という', 'もの', 'を', '見る', 'た'], ['しかも', 'あと', 'で', '聞く', 'と', 'それ', 'は', '書生', 'という', '人間', '中', 'で', '一番', '｜', '獰悪', 'だ', '種族', 'だ', 'ある', 'た', 'そう', 'だ'], ['この', '書生', 'という', 'の', 'は', '時々', '我々', 'を', '捕える', 'て', '煮る', 'て', '食う', 'という', '話', 'だ', 'ある'], ['しかし', 'その', '当時', 'は', '何', 'という', '考', 'も', 'ない', 'た', 'から', '別段', '恐い', 'いとも', '思う', 'ない', 'た'], ['ただ', '彼', 'の', '掌', 'に', '載せる', 'られる', 'て', 'スー', 'と', '持ち上げる', 'られる', 'た', '時', '何だか', 'フワフワ', 'する', 'た', '感じ', 'が', 'ある', 'た', 'ばかり', 'だ', 'ある'], ['掌', 'の', '上', 'で', '少し', '落ちつく', 'て', '書生', 'の', '顔', 'を', '見る', 'た', 'の', 'が', 'いわゆる', '人間', 'という', 'もの', 'の', '見る', '始', 'だ', 'ある', 'う']]


In [25]:
urllib.request.urlretrieve('http://www.cl.ecei.tohoku.ac.jp/resources/sent_lex/wago.121808.pn', 'wago.121808.pn')

('wago.121808.pn', <http.client.HTTPMessage at 0x7fc240ad12b0>)

In [26]:
wago = pd.read_csv('wago.121808.pn', header=None, sep='\t')
wago.head(3)

,0,1
0,ネガ（経験）,あがく
1,ネガ（経験）,あきらめる
2,ネガ（経験）,あきる


In [27]:
#　単語とスコアを対応させる辞書を作成
word2score = {}
values = {'ポジ（経験）': 1, 'ポジ（評価）': 1, 'ネガ（経験）': -1, 'ネガ（評価）': -1}
for word, label in zip(wago.loc[:, 1], wago.loc[:, 0]):
    word2score[word] = values[label]

In [28]:
list(word2score.items())[:3]

[('あがく', -1), ('あきらめる', -1), ('あきる', -1)]

In [29]:
scores = []
for words in words_list:
    score = 0
    #  単語が辞書に現れていればスコアを加算
    for word in words:
        if word in word2score:
            score += word2score[word]
    scores.append(score)

In [30]:
scores_df = pd.DataFrame({'sentence': sentences[:-1], 'score': scores}, columns=['sentence', 'score'])
scores_df.head(5)

,sentence,score
0,一吾輩は猫である,0
1,名前はまだ無い,0
2,どこで生れたかとんと見当がつかぬ,0
3,何でも薄暗いじめじめした所でニャーニャー泣いていた事だけは記憶している,-1
4,吾輩はここで始めて人間というものを見た,0


In [31]:
# スコアの降順に並び返る
scores_df_sorted = scores_df.sort_values('score', ascending=False)
# スコアの高い文書5件を抽出
scores_df_sorted.head(5)

,sentence,score
453,「厭きっぽいのじゃない薬が利かんのだ」「それだってせんだってじゅうは大変によく利くよく利くと...,5
1428,四百六十五行から、四百七十三行を御覧になると分ります」「希臘語｜云々はよした方がいい、さも希...,5
3860,美しい？美しくても構わんから、美しい獣と見做せばいいのである,4
5380,精神の修養を主張するところなぞは大に敬服していい」「敬服していいかね,4
6768,これがぬすみ食をするとか、贋札を造るとか云うなら、まだ始末がいいが、音曲は人に隠しちゃ出来な...,3


In [32]:
# スコアの低い5件を抽出
scores_df_sorted.tail(5)

,sentence,score
2045,何か怒っているかも知れんが、怒るのは向が悪るいからで、先方がおとなしくしてさえいれば一身上の...,-3
6618,どうもいつまで行っても柿ばかり食ってて際限がないね」「私もじれったくてね」「君より聞いてる方...,-4
3783,こんな、しつこい、毒悪な、ねちねちした、執念深い奴は大嫌だ,-4
7098,向うがあやまるなら特別、私の方ではそんな慾はありません」「警察が君にあやまれと命じたらどうで...,-4
6687,「古人を待つ身につらき置炬燵と云われた事があるからね、また待たるる身より待つ身はつらいともあ...,-5
